In [1]:
import pandas as pd
import numpy as np
import datasets as ds
import copy
import dotenv
import json
import os
import warnings
from langchain_google_genai import ChatGoogleGenerativeAI

from agents.core.reflection.agent import ReflexionCoTAgent, SelfReflectionCoTAgent

from agents.core.prompts.gsm8k import (
    GSM8K_FEWSHOT_INSTRUCTION_AGENT
)

from agents.core.reflection.prompts.general import (
    FEWSHOT_REFLECT_INSTRUCTION_NEGATIVE
)
from agents.core.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_AGENT_COT
)

from agents.core.reflection.prompts.gsm8k import (
    GSM8K_FEWSHOT_REFLEXION_EXAMPLES_MIXED,
    GSM8K_FEWSHOT_SELFREFLECTION_EXAMPLES_MIXED
)

from agents.core.reflection.reflect import (
    ReflexionCoTReflector
)

from experiments.utils.logging import log_response

warnings.filterwarnings('ignore')
dotenv.load_dotenv()

total_rows = 200
prompt_examples = "negative_prompt_mixed_examples"

c:\Users\johns\Documents\GitHub\agential\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_stats(prompt_examples):
    # Define the multi-index structure for detailed stats (previously total_stats structure)
    index = pd.MultiIndex.from_tuples([
        (pd.NA, 'cot_1', pd.NA, 'correct'),
        (pd.NA, 'cot_1', pd.NA, 'incorrect'),

        (prompt_examples, 'selfreflection_cot_2', pd.NA, 'correct_state_1'),
        (prompt_examples, 'selfreflection_cot_2', pd.NA, 'correct_state_2'),
        (prompt_examples, 'selfreflection_cot_2', pd.NA, 'incorrect_state_1'),
        (prompt_examples, 'selfreflection_cot_2', pd.NA, 'incorrect_state_2'),

        (prompt_examples, 'selfreflection_cot_2', 'start_to_final', 'correct_to_incorrect'),
        (prompt_examples, 'selfreflection_cot_2', 'start_to_final', 'incorrect_to_correct'),
        (prompt_examples, 'selfreflection_cot_2', 'start_to_final', 'incorrect_to_incorrect'),
        (prompt_examples, 'selfreflection_cot_2', 'start_to_final', 'correct_to_correct'),

        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'correct_state_1'),
        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'correct_state_2'),
        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'correct_state_3'),
        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'incorrect_state_1'),
        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'incorrect_state_2'),
        (prompt_examples, 'selfreflection_cot_3', pd.NA, 'incorrect_state_3'),

        (prompt_examples, 'selfreflection_cot_3', 'start_to_final', 'correct_to_incorrect'),
        (prompt_examples, 'selfreflection_cot_3', 'start_to_final', 'incorrect_to_correct'),
        (prompt_examples, 'selfreflection_cot_3', 'start_to_final', 'incorrect_to_incorrect'),
        (prompt_examples, 'selfreflection_cot_3', 'start_to_final', 'correct_to_correct'),

        (prompt_examples, 'reflexion_cot_2', pd.NA, 'correct_state_1'),
        (prompt_examples, 'reflexion_cot_2', pd.NA, 'correct_state_2'),
        (prompt_examples, 'reflexion_cot_2', pd.NA, 'incorrect_state_1'),
        (prompt_examples, 'reflexion_cot_2', pd.NA, 'incorrect_state_2'),

        (prompt_examples, 'reflexion_cot_2', 'start_to_final', 'correct_to_incorrect'),
        (prompt_examples, 'reflexion_cot_2', 'start_to_final', 'incorrect_to_correct'),
        (prompt_examples, 'reflexion_cot_2', 'start_to_final', 'incorrect_to_incorrect'),
        (prompt_examples, 'reflexion_cot_2', 'start_to_final', 'correct_to_correct'),

        (prompt_examples, 'reflexion_cot_3', pd.NA, 'correct_state_1'),
        (prompt_examples, 'reflexion_cot_3', pd.NA, 'correct_state_2'),
        (prompt_examples, 'reflexion_cot_3', pd.NA, 'correct_state_3'),
        (prompt_examples, 'reflexion_cot_3', pd.NA, 'incorrect_state_1'),
        (prompt_examples, 'reflexion_cot_3', pd.NA, 'incorrect_state_2'),
        (prompt_examples, 'reflexion_cot_3', pd.NA, 'incorrect_state_3'),

        (prompt_examples, 'reflexion_cot_3', 'start_to_final', 'correct_to_incorrect'),
        (prompt_examples, 'reflexion_cot_3', 'start_to_final', 'incorrect_to_correct'),
        (prompt_examples, 'reflexion_cot_3', 'start_to_final', 'incorrect_to_incorrect'),
        (prompt_examples, 'reflexion_cot_3', 'start_to_final', 'correct_to_correct'),
    ], names=['prompt_examples', 'method', 'transition', 'result_type'])

    # Initialize the base stats DataFrame with the multi-index structure
    stats_df = pd.DataFrame(0, index=index, columns=['value']).reset_index()

    return stats_df


def save_stats(new_stats, stats_filename='stats.csv'):
    """
    Overwrites the stats file with new stats.

    Arguments:
    - new_stats (pd.DataFrame): The new stats to write to the file.
    - stats_filename (str): The filename of the stats file.
    """
    try:
        # Overwrite the existing file or create a new one
        new_stats.to_csv(stats_filename, index=False)
        print(f"Stats successfully saved to {stats_filename}.")

    except PermissionError:
        print(f"Permission denied: Unable to write to {stats_filename}.")
        print("Please ensure the file is not open in another application and you have write permissions.")

    except Exception as e:
        print(f"An error occurred while saving stats: {e}")


def get_general_stats(stats_df):
    general_stats_df = stats_df.groupby(['prompt_examples', 'method', 'transition', 'result_type']).agg({
        'value': 'sum'  # Sum up the values for each group
    }).reset_index()

    return general_stats_df


def update_stats(stats_df, prompt_examples, method, trial_num=None, state=None):
    """
    Helper function to update stats_df based on provided parameters.

    Arguments:
    - stats_df: The DataFrame storing all statistics.
    - prompt_examples: The specific prompt examples used in this context.
    - method: The method being used (e.g., 'cot_1', 'selfreflection_cot_3').
    - trial_num: The trial number (1, 2, or 3). Set to None for single-trial methods like 'cot_1'.
    - state: 'correct' or 'incorrect'.
    """
    if method == 'cot_1':
        # For single-trial methods, use 'correct' or 'incorrect' as result_type
        if state not in ['correct', 'incorrect']:
            raise ValueError("State must be 'correct' or 'incorrect' for 'cot_1'")
        result_type = state

        # Define the condition to locate the correct row in stats_df
        condition = (
            (stats_df['method'] == method) &
            (stats_df['result_type'] == result_type)
        )
    else:
        # For multi-trial methods, use 'correct_state_x' or 'incorrect_state_x'
        if trial_num is None or state is None:
            raise ValueError("Both trial_num and state must be provided for multi-trial methods")
        result_type = f'{state}_state_{trial_num}'

        # Define the condition to locate the correct row in stats_df
        condition = (
            (stats_df['prompt_examples'] == prompt_examples) &
            (stats_df['method'] == method) &
            (stats_df['transition'].isna()) &  # No transition for state rows
            (stats_df['result_type'] == result_type)
        )

    # Increment the 'value' by 1 for matching rows
    if stats_df.loc[condition, 'value'].empty:
        print(f"Warning: No matching row found for method '{method}', result_type '{result_type}'.")
    else:
        stats_df.loc[condition, 'value'] += 1


def determine_transition(prev_correct, current_correct):
    """
    Determines the transition type based on previous and current correctness.
    
    Arguments:
    - prev_correct: Boolean indicating if the previous trial was correct.
    - current_correct: Boolean indicating if the current trial was correct.
    
    Returns:
    - A string representing the transition type (e.g., 'correct_to_incorrect', 'incorrect_to_correct').
    """
    if prev_correct and not current_correct:
        return 'correct_to_incorrect'
    elif not prev_correct and current_correct:
        return 'incorrect_to_correct'
    elif not prev_correct and not current_correct:
        return 'incorrect_to_incorrect'
    else:
        return 'correct_to_correct'


def process_transitions(stats_df, output_list, prompt_examples, method, transition_labels, max_trials):
    """
    Processes transitions for a given output list and updates stats_df accordingly.

    Arguments:
    - stats_df: The DataFrame storing all statistics.
    - output_list: List of output objects with 'is_correct' attribute.
    - num_people, num_days, duration: Group-specific metadata.
    - prompt_examples: The specific prompt examples used.
    - method: The method being used (e.g., 'selfreflection_cot_3').
    - transition_labels: The transitions to process (e.g., ['start_to_final']).
    - max_trials: Maximum number of trials for the method (2 or 3).
    """
    # Determine actual number of trials performed
    actual_trials = len(output_list)
    
    # Extract trial states
    trial_states = ['correct' if output.is_correct else 'incorrect' for output in output_list]
    
    # Pad the trial_states with the last state if method stopped early
    if actual_trials < max_trials:
        if actual_trials > 0:
            last_state = trial_states[-1]
        else:
            last_state = 'incorrect'  # Default to incorrect if no trials were performed
        trial_states.extend([last_state] * (max_trials - actual_trials))
    
    # Update the 'correct_state_x' and 'incorrect_state_x' counts
    for trial_num, state in enumerate(trial_states, start=1):
        update_stats(
            stats_df, 
            prompt_examples, 
            method, 
            trial_num=trial_num, 
            state=state
        )
    
    # Handle transitions based on transition_labels
    for transition_label in transition_labels:
        if transition_label == 'start_to_final':
            if len(trial_states) >= 2:
                # Transition from first to final trial
                prev_correct = trial_states[0] == 'correct'
                final_correct = trial_states[-1] == 'correct'
                transition = determine_transition(prev_correct, final_correct)
                # Update the 'start_to_final' transition
                condition = (
                    (stats_df['prompt_examples'] == prompt_examples) &
                    (stats_df['method'] == method) &
                    (stats_df['transition'] == 'start_to_final') &
                    (stats_df['result_type'] == transition)
                )
                # Increment the 'value' by 1 for matching transition rows
                if stats_df.loc[condition, 'value'].empty:
                    print(f"Warning: No matching transition row found for method '{method}', transition '{transition}'.")
                stats_df.loc[condition, 'value'] += 1


In [3]:
llm_2 = ChatGoogleGenerativeAI(google_api_key="", model="gemini-1.5-flash", temperature=0.0)
llm_3 = ChatGoogleGenerativeAI(google_api_key="", model="gemini-1.5-flash", temperature=0.0)
llm_4 = ChatGoogleGenerativeAI(google_api_key="", model="gemini-1.5-flash", temperature=0.0)
llm_5 = ChatGoogleGenerativeAI(google_api_key="", model="gemini-1.5-flash", temperature=0.0)

In [4]:
gsm8k_ds = ds.load_dataset('gsm8k', 'main', split="test")
downsampled_ds = gsm8k_ds.shuffle(seed=40).select(range(total_rows))

stats_df = create_stats(prompt_examples)

In [5]:
counter = 0
for i, row in enumerate(downsampled_ds):
    counter += 1
    print(f"Processing row {i+1} of {len(downsampled_ds)} out of {total_rows}...")

    question = row["question"] + " Solve the math question."
    key = int(row["answer"].split("####")[1].replace(",", ""))

    ###################################################
    ######### SELF-REFLECTION + COT (2 TRIALS) ########
    ###################################################
    
    agent_2 = SelfReflectionCoTAgent(
        llm=llm_2, 
        benchmark="gsm8k",
        reflector=ReflexionCoTReflector(llm=llm_2),
        max_reflections=2,
        max_trials=2
    )

    out_cot_selfreflect_2 = agent_2.generate(
        question=question,
        key=key,
        examples=GSM8K_FEWSHOT_EXAMPLES_AGENT_COT,
        prompt=GSM8K_FEWSHOT_INSTRUCTION_AGENT,
        reflect_examples=GSM8K_FEWSHOT_SELFREFLECTION_EXAMPLES_MIXED,
        reflect_prompt=FEWSHOT_REFLECT_INSTRUCTION_NEGATIVE,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        max_trials=2,
        patience=2,
        reset=True
    )

    # Save output to file
    with open(f"./logs/{prompt_examples}_selfreflection_cot_2.txt", "a", encoding="utf-8") as f_2:
        f_2.write(f"""
---------------------------------
QUESTION: {question}
KEY: {key}
---------------------------------
REFLECTION COT OUTPUT
---------------------------------
""")
        log_response(f_2, out_cot_selfreflect_2)

    # Determine and update trial states with padding
    method = 'selfreflection_cot_2'
    max_trials = 2
    transition_labels = ['start_to_final']
    process_transitions(
        stats_df,
        out_cot_selfreflect_2,
        prompt_examples,
        method,
        transition_labels,
        max_trials
    )

    ###################################################
    ######### SELF-REFLECTION + COT (3 TRIALS) ########
    ###################################################

    agent_3 = SelfReflectionCoTAgent(
        llm=llm_3,
        benchmark="gsm8k",
        reflector=ReflexionCoTReflector(llm=llm_3),
        max_reflections=3,
        max_trials=3
    )

    out_cot_selfreflect_3 = agent_3.generate(
        question=question,
        key=key,
        examples=GSM8K_FEWSHOT_EXAMPLES_AGENT_COT,
        prompt=GSM8K_FEWSHOT_INSTRUCTION_AGENT,
        reflect_examples=GSM8K_FEWSHOT_SELFREFLECTION_EXAMPLES_MIXED,
        reflect_prompt=FEWSHOT_REFLECT_INSTRUCTION_NEGATIVE,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        max_trials=3,
        patience=3,
        reset=True
    )

    # Save output to file
    with open(f"./logs/{prompt_examples}_selfreflection_cot_3.txt", "a", encoding="utf-8") as f_3:
        f_3.write(f"""
---------------------------------
QUESTION: {question}
KEY: {key}
---------------------------------
REFLECTION COT OUTPUT
---------------------------------
""")
        log_response(f_3, out_cot_selfreflect_3)

    # Determine and update trial states with padding
    method = 'selfreflection_cot_3'
    max_trials = 3
    transition_labels = ['start_to_final']
    process_transitions(
        stats_df,
        out_cot_selfreflect_3,
        prompt_examples,
        method,
        transition_labels,
        max_trials
    )

    ############################################
    ######## REFLEXION + COT (2 TRIALS) ########
    ############################################

    agent_4 = ReflexionCoTAgent(
        llm=llm_4,
        benchmark="gsm8k",
        reflector=ReflexionCoTReflector(llm=llm_4),
        max_reflections=2,
        max_trials=2
    )

    out_cot_reflect_2 = agent_4.generate(
        question=question,
        key=key,
        examples=GSM8K_FEWSHOT_EXAMPLES_AGENT_COT,
        prompt=GSM8K_FEWSHOT_INSTRUCTION_AGENT,
        reflect_examples=GSM8K_FEWSHOT_REFLEXION_EXAMPLES_MIXED,
        reflect_prompt=FEWSHOT_REFLECT_INSTRUCTION_NEGATIVE,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        max_trials=2,
        patience=2,
        reset=True
    )

    # Save output to file
    with open(f"./logs/{prompt_examples}_reflexion_cot_2.txt", "a", encoding="utf-8") as f_4:
        f_4.write(f"""
---------------------------------
QUESTION: {question}
KEY: {key}
---------------------------------
REFLECTION COT OUTPUT
---------------------------------
""")
        log_response(f_4, out_cot_reflect_2)

    # Determine and update trial states with padding
    method = 'reflexion_cot_2'
    max_trials = 2
    transition_labels = ['start_to_final']
    process_transitions(
        stats_df,
        out_cot_reflect_2,
        prompt_examples,
        method,
        transition_labels,
        max_trials
    )

    ############################################
    ######## REFLEXION + COT (3 TRIALS) ########
    ############################################

    agent_5 = ReflexionCoTAgent(
        llm=llm_5,
        benchmark="gsm8k",
        reflector=ReflexionCoTReflector(llm=llm_5),
        max_reflections=3,
        max_trials=3
    )

    out_cot_reflect_3 = agent_5.generate(
        question=question,
        key=key,
        examples=GSM8K_FEWSHOT_EXAMPLES_AGENT_COT,
        prompt=GSM8K_FEWSHOT_INSTRUCTION_AGENT,
        reflect_examples=GSM8K_FEWSHOT_REFLEXION_EXAMPLES_MIXED,
        reflect_prompt=FEWSHOT_REFLECT_INSTRUCTION_NEGATIVE,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        max_trials=3,
        patience=3,
        reset=True
    )

    # Save output to file
    with open(f"./logs/{prompt_examples}_reflexion_cot_3.txt", "a", encoding="utf-8") as f_5:
        f_5.write(f"""
---------------------------------
QUESTION: {question}
KEY: {key}
---------------------------------
REFLECTION COT OUTPUT
---------------------------------
""")
        log_response(f_5, out_cot_reflect_3)

    # Determine and update trial states with padding
    method = 'reflexion_cot_3'
    max_trials = 3
    transition_labels = ['start_to_final']
    process_transitions(
        stats_df,
        out_cot_reflect_3,
        prompt_examples,
        method,
        transition_labels,
        max_trials
    )

    print(f"Processing row {i+1} of {len(downsampled_ds)} out of {total_rows}... DONE")
    save_stats(stats_df, stats_filename=f'./results/{prompt_examples}_stats.csv')


Processing row 1 of 200 out of 200...


C:\Users\johns\Documents\GitHub\agential\agents\core\reflection\functional.py:117: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  out = llm(


Processing row 1 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 2 of 200 out of 200...
Processing row 2 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 3 of 200 out of 200...
Processing row 3 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 4 of 200 out of 200...
Processing row 4 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 5 of 200 out of 200...
Processing row 5 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 6 of 200 out of 200...
Processing row 6 of 200 out of 200... DONE
Stats successfully saved to ./results/negative_prompt_mixed_examples_stats.csv.
Processing row 7 of 200 out of 200...
Processing row 7 of 200 out of 200